# TOMM7
Pathogenic variants in TOMM7 have been pobserved in individuals with [Garg-Mishra progeroid syndrome ](https://omim.org/entry/620601).

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.56


In [2]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_excel("input/TOMM7_individuals.xlsx")
df.head(2)

,PMID,title,individual_id,comment,disease_id,disease_label,transcript,allele_1,allele_2,variant.comment,...,Hypotonia,Increased circulating lactate dehydrogenase concentration,Persistent open anterior fontanelle,Thin ribs,Ovoid vertebral bodies,Platyspondyly,Slender long bone,Coxa valga,Slender metacarpals,Microvesicular hepatic steatosis
0,str,str,str,optional str,str,str,str,str,str,optional str,...,HP:0001252,HP:0025435,HP:0004474,HP:0000883,HP:0003300,HP:0000926,HP:0003100,HP:0002673,HP:0006236,HP:0001414
1,PMID:36282599,Autosomal recessive progeroid syndrome due to homozygosity for a TOMM7 variant,21-year-old man,NaN,OMIM:620601,Garg-Mishra progeroid syndrome,NM_019059.5,c.86C>T,c.86C>T,NP_061932.1:p.Pro29Leu,...,na,na,na,na,na,na,na,na,na,na


In [4]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by)
individuals = encoder.get_individuals()

Created encoders for 42 fields


In [5]:
TOMM7_transcript = "NM_019059.5"
vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          gene_symbol="TOMM7",
                          transcript=TOMM7_transcript,
                          allele_1_column_name="allele_1",
                          allele_2_column_name="allele_2")

[INFO] encoding variant "c.86C>T"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_019059.5%3Ac.86C>T/NM_019059.5?content-type=application%2Fjson
[INFO] encoding variant "c.73T>C"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_019059.5%3Ac.73T>C/NM_019059.5?content-type=application%2Fjson


In [6]:
vmanager.to_summary()

,status,count,alleles
0,mapped,2,"c.86C>T, c.73T>C"
1,unmapped,0,


In [7]:
vmanager.add_variants_to_individuals(individuals)
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1,
                                allelic_requirement=AllelicRequirement.BI_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [8]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
21-year-old man (MALE; P21Y),Garg-Mishra progeroid syndrome (OMIM:620601),NM_019059.5:c.86C>T (homozygous),Postnatal growth retardation (HP:0008897); Short stature (HP:0004322); Reduced visual acuity (HP:0007663); Secondary microcephaly (HP:0005484); Triangular face (HP:0000325); Broad forehead (HP:0000337); Prominent nasal bridge (HP:0000426); Bulbous nose (HP:0000414); Micrognathia (HP:0000347); Dental crowding (HP:0000678); Nystagmus (HP:0000639); High hypermetropia (HP:0008499); Microphthalmia (HP:0000568); Cafe-au-lait spot (HP:0000957); Sparse hair (HP:0008070)
affected individual (MALE; P2Y7M),Garg-Mishra progeroid syndrome (OMIM:620601),NM_019059.5:c.73T>C (homozygous),Postnatal growth retardation (HP:0008897); Short stature (HP:0004322); Nystagmus (HP:0000639); Narrow chest (HP:0000774); Broad palm (HP:0001169); Small nail (HP:0001792); Hypotonia (HP:0001252); Increased circulating lactate dehydrogenase concentration (HP:0025435); Persistent open anterior fontanelle (HP:0004474); Thin ribs (HP:0000883); Ovoid vertebral bodies (HP:0003300); Platyspondyly (HP:0000926); Slender long bone (HP:0003100); Coxa valga (HP:0002673); Slender metacarpals (HP:0006236); Microvesicular hepatic steatosis (HP:0001414)


In [9]:
encoder.output_individuals_as_phenopackets(individual_list=individuals)

We output 2 GA4GH phenopackets to the directory phenopackets


# HPOA file

In [10]:
ingestor = PhenopacketIngestor(indir="phenopackets")
ppkt_d = ingestor.get_phenopacket_dictionary()
ppkt_list = list(ppkt_d.values())

[pyphetools] Ingested 2 GA4GH phenopackets.


In [11]:
builder = HpoaTableBuilder(phenopacket_list=ppkt_list)
PMID = "PMID:36282599"
hpoa_table_creator = builder.autosomal_recessive(PMID).build()

We found a total of 28 unique HPO terms
Extracted disease: Garg-Mishra progeroid syndrome (OMIM:620601)


In [12]:
df = hpoa_table_creator.get_dataframe()
df.head()

,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620601,Garg-Mishra progeroid syndrome,HP:0008897,Postnatal growth retardation,,,1/1,,,,,PMID:36299998,PCS,ORCID:0000-0002-0736-9199
1,OMIM:620601,Garg-Mishra progeroid syndrome,HP:0004322,Short stature,,,1/1,,,,,PMID:36299998,PCS,ORCID:0000-0002-0736-9199
2,OMIM:620601,Garg-Mishra progeroid syndrome,HP:0000639,Nystagmus,,,1/1,,,,,PMID:36299998,PCS,ORCID:0000-0002-0736-9199
3,OMIM:620601,Garg-Mishra progeroid syndrome,HP:0000774,Narrow chest,,,1/1,,,,,PMID:36299998,PCS,ORCID:0000-0002-0736-9199
4,OMIM:620601,Garg-Mishra progeroid syndrome,HP:0001169,Broad palm,,,1/1,,,,,PMID:36299998,PCS,ORCID:0000-0002-0736-9199


In [13]:
hpoa_table_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620601.tab
